In [18]:
import logging
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

data=pd.read_csv("feature_frames.csv")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
order_sizes = data.groupby('order_id').size()
orders_with_5_or_more_products = order_sizes[order_sizes >= 5].index
filtered_data = data[data['order_id'].isin(orders_with_5_or_more_products)]
filtered_data = data[data['user_order_seq'] > 5]
df = pd.DataFrame(filtered_data)
df['created_at'] = pd.to_datetime(df['created_at'])
df['order_date'] = pd.to_datetime(df['order_date'])

def get_season(month):
    if month in [12, 1, 2]:
        return 1  # winter
    elif month in [3, 4, 5]:
        return 2  # spring
    elif month in [6, 7, 8]:
        return 3  # summer
    else:
        return 4  # Outumn


df['season'] = df['order_date'].dt.month.apply(get_season)
df['day_of_week'] = df['order_date'].dt.dayofweek
df['is_weekend'] = df['order_date'].dt.dayofweek >= 5
df['hour_of_day'] = df['order_date'].dt.hour
variables_categoricas = ['product_type', 'vendor']
variables_binarias = ['ordered_before', 'abandoned_before', 'active_snoozed', 'set_as_regular','is_weekend']

for columna in variables_categoricas:
    frecuencias = df[columna].value_counts().to_dict()
    df[columna + '_freq'] = df[columna].map(frecuencias)

for columna in variables_binarias:
    df[columna] = df[columna].astype(int)

X = df.drop(['variant_id', 'order_id', 'user_id', 'product_type','vendor','created_at', 'order_date', 'outcome'], axis=1)
X = pd.concat([X, df[['day_of_week', 'is_weekend', 'hour_of_day']]], axis=1)
y = df['outcome']

logging.info("X first rows:\n%s", X.head().to_string())
   







2024-02-15 15:21:23,990 - INFO - X first rows:
    user_order_seq  ordered_before  abandoned_before  active_snoozed  set_as_regular  normalised_price  discount_pct  global_popularity  count_adults  count_children  count_babies  count_pets  people_ex_baby  days_since_purchase_variant_id  avg_days_to_buy_variant_id  std_days_to_buy_variant_id  days_since_purchase_product_type  avg_days_to_buy_product_type  std_days_to_buy_product_type  season  day_of_week  is_weekend  hour_of_day  product_type_freq  vendor_freq  day_of_week  is_weekend  hour_of_day
7               13               0                 0               0               0          0.081052      0.053512           0.038462           2.0             0.0           0.0         0.0             2.0                            33.0                        42.0                   31.134053                              30.0                          30.0                      24.27618       4            1           0            0            